# Produccion electrica en base a la metereología

La idea principal del trabajo es ser capaces de predecir como va a afectar a que aumenten la produccion electrica de energias renovables las precipitaciones, el viento y las horas de sol.
Como las horas de sol y el viento son fenomenos cuya causa efecto es prácticamente inmediato vamos a centrarnos en la produccion de energia hidroelectrica.

En base a esto tenemos varias cuestiones que queremos despejar:
- Que relación hay entre precipitaciones y el aumento de la generación de energia eléctrica hidráulica.
- Que relación hay entre horas de sol, temperatura y el aumento de la generación de energia eléctrica sólar.
- Que relación hay entre viento y el aumento de la generación de energia eléctrica eólica, esta relación esta condicionada por la temperatura, percipitaciones o horas de sol.
- Cuantos días tarda en aumentar la generación eléctrica de fuentes de energía renovables en función de los fenomenos metereológicos.
- Dados una prediccion meteorologica que valores de generación eléctrica tendremos para una fecha determinada.



Como premisas partimos de :
- Vamos a considerar solo el poll de energia que proporciona Red Electrica de España (REE)
- Vamos a considerar que las empresas no trabajan bajo mala praxis y que intentan optimizar el uso de energias renovables.
- Debido a la falta de datos a nivel diario de REE por provincia o comunidades autonomas, voy a centrar el analisis a nivel de sistema eléctrico (Peninsula, Baleares, Canarias ,Ceuta y Melilla).

Como origenes de datos para el estudio vamos a utilizar los datos proporcionados por:
- Información de REE (https://www.ree.es/es/apidatos) obtenida mediante su API.
- Datos proporcionados por aemet(https://opendata.aemet.es/centrodedescargas/inicio), vamos a utilizar la libreria  aemet desarrollada por Pablo Moreno (https://pypi.org/project/python-aemet/).

Requisitos para la ejecución del notebook:

Como requisitos para la ejecución del proyecto es necesario la instalación de la libreria python Aemet(pip install python-aemet) e instalar la libreria request.

Además las versiones de cada libreria utilizada en este proyecto son:

El modelo ha utilizar al tratarse de una prediccion númerica y no de obtener una etiqueta, sera una regresión.
Para poder llevar a cabo esa regresion se han pasado los valores de fechas a númericos, y se ha creado una columna por cada tipo de energia, creando varias variables objetivo.



In [1]:
from aemet import Aemet,Estacion
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import json
import requests
import time

# Leemos la clave de la API de AEMET desde un fichero ubicado en la misma ruta que este notebook
with open('API_KEY_AEMET','r') as file:
    API_KEY_AE=file.read()

    

###  Lectura datos AEMET

Para la lectura de datos metereólogicos, vamos a utilizar la libreria aemet, de la que utilizaremos los metodos de las clases Aemet y Estacion para obtener los datos a nivel diario de cada estación meterologica para un rago de fechas, de las provincias que nos interesan para el estudio.

In [2]:
# Obtenemos el json de estaciones de mediciones de aemet 
info_estaciones=Estacion.get_estaciones(API_KEY_AE)

# Creamos un objeto Aemet para usar los metodos de la libreria aemet
aemet=Aemet(API_KEY_AE)

In [3]:
from tqdm import tqdm_notebook as tqdm

# Definimos funciones que vamos a utilizar para leer los datos de AEMET
def estaciones_prov (prov,lista_estaciones):
    '''Dada una lista de provincias y un json de estaciones de aemet. 
    Obtiene una lista de los ID de las estaciones de esa provincia.'''
    lista_id=[]
    prov=list(map(str.upper, prov))
    for estacion in tqdm(lista_estaciones):
        if estacion['provincia'] in prov:
            lista_id.append(estacion['indicativo'])
    return lista_id


def lectura_diaria_json(date_ini,date_end,estaciones):
    '''Dado un json de la clase Estacion de la libreria Aemet, y fechas de inicio y fin:
    Obtenemos los datos climatologicos entre las dos fechas para todas las estaciones de manera diaria
    Si la fecha de inicio es anterior a 2016, se cambia a 2016-01-01, para evitar errores.
    '''
    valores_diarios=[]
    valores_error=[]
    if date_ini[0:4]<'2016-01':
        date_ini="2016-01-01T00:00:00UTC"
    if date_ini>date_end:
        print('Valores no válidos, fecha de inicio mayor que la fecha de fin')
        return valores_diarios;
    for element in tqdm(estaciones):
        try:
            valores_estacion=aemet.get_valores_climatologicos_diarios(date_ini,date_end,element['indicador'])
            if type(valores_estacion)!=dict:
                valores_diarios.extend(valores_estacion)
        except:
            time.sleep(56) # para evitar errores por nº de lecturas por minuto.
            #Volvemos a intentar leer el dato que ha lanzado la excepcion.
            try:
                valores_estacion=aemet.get_valores_climatologicos_diarios(date_ini,date_end,element['indicador'])
                if type(valores_estacion)!=dict:
                    valores_diarios.extend(valores_estacion)
            except:
                print('Valor no encontrado')
                
    return valores_diarios;


def lectura_diaria_lista(date_ini,date_end,lista_estaciones):
    '''Dado una lista de id de estaciones de aemet, y fechas de inicio y fin:
    Obtenemos los datos climatologicos entre las dos fechas para todas las estaciones de manera diaria
    Si la fecha de inicio es anterior a 2016, se cambia a 2016-01-01, para evitar errores.
    '''
    valores_diarios=[]
    if date_ini[0:4]<'2016-01':
        date_ini="2016-01-01T00:00:00UTC"
    if date_ini>date_end:
        print('Valores no válidos, fecha de inicio mayor que la fecha de fin')
        return valores_diarios;
    for element in tqdm(lista_estaciones):
        try:
            valores_estacion=aemet.get_valores_climatologicos_diarios(date_ini,date_end,element)
            if type(valores_estacion)!=dict:
                valores_diarios.extend(valores_estacion)
        except:
            time.sleep(56) # para evitar errores por nº de lecturas.
            try:
                valores_estacion=aemet.get_valores_climatologicos_diarios(date_ini,date_end,element)
                if type(valores_estacion)!=dict:
                    valores_diarios.extend(valores_estacion)
            except:
                print('Valor no encontrado')
                
    return valores_diarios;

In [4]:
# Parametros para lectura de datos de AEMET

provincias=['Ceuta','Melilla','Illes Balears','Las Palmas','Sta. Cruz de Tenerife']

date_ini="2020-12-01T00:00:00UTC"
date_end="2020-12-30T23:59:59UTC"

id_estaciones=estaciones_prov(provincias,info_estaciones)

# Pasamos los datos ha dataframes para su procesado y limpieza
estaciones=pd.DataFrame(info_estaciones)
df_weather=pd.DataFrame(lectura_diaria_lista(date_ini,date_end,id_estaciones),dtype=str)

### Limpieza datos AEMET

In [5]:
# Creo una funcion que pase los indicadores a float y rellene los valores vacios por la temperatura media de las estaciones de esa provincia
def rellena_nulos_provincia(df,cols):
    # Defino un DataFrame vacio para acumular el resultado
    df_all=pd.DataFrame()
    
    # Hago un bucle para cada provincia del DataFrame de entrada
    for prov in df['provincia'].unique():
        df_prov=df[df['provincia']==prov].copy()
        
        #Para cada elemento de las columnas que nos interesan reemplazo , por ., paso a numerico y relleno los nulos por la media de la provincia
        for element in cols:
            df_prov[element]=df_prov[element].str.replace(',', '.')
            df_prov[element]=pd.to_numeric(df_prov[element],errors='coerce')
            df_prov[element].fillna(df_prov[element].mean(skipna=True),inplace=True)
        df_all=df_all.append(df_prov)
    return df_all

In [6]:
# Elimino las columnas que no me interesan
df_weather.drop(columns=['horaPresMax','horaPresMin','horaracha','dir','horatmin','horatmax'],inplace=True)

# Limpio de nulos la muestra
df_weather=rellena_nulos_provincia(df_weather,df_weather.columns[5:])


# Análisis de la muestra

1 Evolución de temperaturas

2 Correlaciones

3 Evolución de precipitaciones

4 Datos estacionales

5 Localizar datos anomalos


## Lectura de los datos de REE

Para la lectura de los datos de REE voy a utilizar la libreria python requests para a traves de su API, obtener la generación de electricidad en , para cada tipo de energia Electrica.

Una vez leidos los datos de la API, guardo los datos en formato json en un fichero. para no tener que repetir las consultas y poder trabajar sin conexion.

La estrutura de los datos leidos de REE es la siguiente:


Por lo que vamos a almacenar 2 ficheros:
    - Renovables
    - No Renovables



In [7]:
# Leemos las regiones de ree obtenidas desde (https://www.ree.es/es/apidatos) desde un fichero ubicado en la misma ruta que este notebook
region_ree=pd.read_csv('REGION_REE',header=0,index_col='Region')

# Me quedo solo con los distintos sistemas electricos existentes
region_system=region_ree[region_ree['geo_limit']!='ccaa']

region_system

,geo_limit,geo_id
Region,,
peninsular,peninsular,8741
canarias,canarias,8742
baleares,baleares,8743
ceuta,ceuta,8744
melilla,melilla,8745


In [8]:
geo_limit=region_system[region_system['geo_id']==8745]['geo_limit']
geo_limit[0]

'melilla'

In [10]:
# Obtenemos los datos de REE a traves de su API.

def lectura_ree_electric_system(d_inicio,d_fin,geo_id):
    
    # meter esto en una funcion con su try-exception    
    
    geo_limit=region_system[region_system['geo_id']==geo_id]['geo_limit']
    
    parametros={'start_date':date_ini,
            'end_date':date_end,
            'time_trunc':'day',
            'geo_trunc':'electric_system',
            'geo_limit':geo_limit[0],
            'geo_ids':geo_id}
    
    URL_GEN='https://apidatos.ree.es/es/datos/generacion/estructura-generacion'

    ree_gen=requests.get(URL_GEN,params=parametros)
    
    df_ree=pd.DataFrame()
    for i in range(20):
        try:
            df=pd.json_normalize(ree_gen.json()['included'][i]['attributes'],meta=['title','type'],record_path=['values'])
            df['system']=geo_limit[0]
            df_ree=df_ree.append(df)
        except:
            pass #Cuando no hay datos para mas tecnologías
    df_ree.reset_index(inplace=True,drop=True)
    
    return df_ree

In [11]:
df_ree_system=pd.DataFrame()
for electric_system in tqdm(region_system['geo_id']):
    df_ree_system=df_ree_system.append(lectura_ree_electric_system(date_ini,date_end,electric_system))
df_ree_system   

,value,percentage,datetime,title,type,system
0,79704.735,0.114638,2020-12-01T00:00:00.000+01:00,Hidráulica,Renovable,peninsular
1,78110.077,0.100441,2020-12-02T00:00:00.000+01:00,Hidráulica,Renovable,peninsular
2,90336.334,0.118152,2020-12-03T00:00:00.000+01:00,Hidráulica,Renovable,peninsular
3,85769.482,0.105608,2020-12-04T00:00:00.000+01:00,Hidráulica,Renovable,peninsular
4,69258.852,0.092469,2020-12-05T00:00:00.000+01:00,Hidráulica,Renovable,peninsular
...,...,...,...,...,...,...
123,544.484,1.000000,2020-12-27T00:00:00.000+01:00,Generación total,Generación total,melilla
124,568.881,1.000000,2020-12-28T00:00:00.000+01:00,Generación total,Generación total,melilla
125,585.858,1.000000,2020-12-29T00:00:00.000+01:00,Generación total,Generación total,melilla
126,589.612,1.000000,2020-12-30T00:00:00.000+01:00,Generación total,Generación total,melilla
